# Multilayer Perceptum (MLP) <a name="multilayer"></a>


We are going to use **Multilayer Perceptron (MLP)** because it is a flexible neural network architecture. MLPs are great for solving **classification problems**

First, we will define the model architecture, this step consists in:
- **Number of layers**
- **Number of neurons of each layer**
- **Choice of the activation functions**


Then we'll perform the training strategy, this strategy includes:
- **Optimizer**
- **Learning hyperparameters** (e.g., learning rate, mini-batch size, number of epochs, etc.)
- **Regularization techniques to adopt** (e.g., early stopping, weight regularization, dropout, data augmentation, etc.)
- **Possibility of using transfer learning**

The network works by processing data through **multiple layers**, with each layer learning to capture different features of the input data.

### Model architecture definition <a name="architecture"></a>
[[go back to the top]](#multilayer)

For the architecture of our MLP model we need, as mentioned above, the number of layers, neurons, and choose the activation functions such as relu, softmax and Tanh for example.


The following table defines the our model architecture:

| <span style="color: #C70039;">**Architecture**</span> | <span style="color: #C70039;">**Options**</span>  |
|-----------------------------------------------------|-------------------------------------------------------|
| <span style="color: #00bfae;">**hidden_units**</span> | [[128, 64], [256, 128, 64], [64, 32]]               |
| <span style="color: #00bfae;">**Activation functions**</span> | ['relu', 'relu', 'tanh']                                  |   

- **hidden_units** consists in the number of layers and the number of each neurons of each layer, for example in this case [256, 128, 64], it defines 3 layers with 256, 128 and 64 neurons, respectively.


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import itertools
from pathos.multiprocessing import Pool

# Classe para definir a arquitetura do modelo MLP
class MLP(tf.keras.Model):
    def __init__(self, input_dim, output_dim, hidden_units):
        super(MLP, self).__init__()
        self.hidden_layers = []
        for units in hidden_units:
            self.hidden_layers.append(tf.keras.layers.Dense(units, activation='relu'))
        self.output_layer = tf.keras.layers.Dense(output_dim, activation='softmax')  

    def call(self, inputs):
        x = inputs
        for layer in self.hidden_layers:
            x = layer(x)
        return self.output_layer(x)
    
    def build(self, input_shape):
        super(MLP, self).build(input_shape)
        # Informar ao TensorFlow as dimensões esperadas das entradas
        self.call(tf.keras.Input(shape=input_shape[1:]))


input_dim = 40  # Exemplo: número de features
output_dim = 10  # Exemplo: número de classes
hidden_units = [128, 64, 32]

model = MLP(input_dim=input_dim, output_dim=output_dim, hidden_units=hidden_units)

# Construir o modelo explicitamente
model.build((None, input_dim))
model.summary()

Model: "mlp_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 128)            │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,914 (62.16 KB)

 Trainable params: 15,914 (62.16 KB)

 Non-trainable params: 0 (0.00 B)

### Training Strategy <a name="training"></a>
[[go back to the top]](#multilayer)

We used **dictionaries** to organize and store different options for **hyperparameters**. This allows us to easily experiment with different configurations and manage the settings efficiently.

To optimize our model, we decided to **update and select the best hyperparameter combination in the first iteration**. This means that in the beginning, we test several combinations of hyperparameters to find the one that performs best. By doing this, we can quickly narrow down the best model for our task, improving the **accuracy** of the predictions.

Additionally, we will use the **ADAM** optimizer, which is a popular choice for training neural networks due to its adaptive learning rate and efficient performance.
We also implemented **early stopping** to prevent overfitting by monitoring the model's performance and halting training when it stops improving.

In this way, the process of **testing and updating** in the first iteration helps us fine-tune the model efficiently, and **selecting the best combination** ensures we are using the most effective settings for our dataset.

---


---

### Hyperparameter Configuration

The following table defines the possible combinations of hyperparameters we tested:

| <span style="color: #C70039;">**Hyperparameter**</span> | <span style="color: #C70039;">**Options**</span>        |
|-----------------------------------------------------|-------------------------------------------------------|
| <span style="color: #00bfae;">**hidden_units**</span> | [[128, 64], [256, 128, 64], [64, 32]]                 |
| <span style="color: #00bfae;">**dropout_rate**</span> | [0.3, 0.5]                                            |
| <span style="color: #00bfae;">**batch_size**</span>   | [32]                                                  |
| <span style="color: #00bfae;">**epochs**</span>       | [20]                                                  |
| <span style="color: #00bfae;">**learning_rate**</span> | [0.001, 0.0001]                                       |

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import itertools
from pathos.multiprocessing import Pool

# Classe MLP
class MLP(tf.keras.Model):
    def __init__(self, input_dim, output_dim, hidden_units, dropout_rate, activations):
        super(MLP, self).__init__()
        self.hidden_layers = []
        for units, activation in zip(hidden_units, activations):
            self.hidden_layers.append(tf.keras.layers.Dense(units, activation=activation))
            self.hidden_layers.append(tf.keras.layers.Dropout(dropout_rate))
        self.output_layer = tf.keras.layers.Dense(output_dim, activation='softmax')  # Classificação multi-classe

    def call(self, inputs):
        x = inputs
        for layer in self.hidden_layers:
            x = layer(x)
        return self.output_layer(x)
    
    
def generate_configs(configurations):
    keys, values = zip(*configurations.items())
    return [dict(zip(keys, v)) for v in itertools.product(*values)]

# Gerar todas as combinações de hiperparâmetros
def generate_activation_configs(hidden_units_list, activation_functions):
    activation_configs = []
    for hidden_units in hidden_units_list:
        # Gerar combinações de funções de ativação do mesmo tamanho das camadas ocultas
        activations = list(itertools.product(activation_functions, repeat=len(hidden_units)))
        activation_configs.extend([(hidden_units, activation) for activation in activations])
    return activation_configs

hidden_units_list = [[128, 64, 32], [256, 128, 64], [64, 32], [512, 256, 128], [256, 128, 64, 32]]
activation_functions = ['relu', 'sigmoid', 'tanh']  # Funções de ativação possíveis

# Combinar hidden_units com ativations
activation_configs = generate_activation_configs(hidden_units_list, activation_functions)


# Função para carregar os dados de um fold específico
def load_fold_data(fold_number, files):
    data = pd.read_csv(files[fold_number])
    labels = data.pop('Label').values  # Extrair os rótulos diretamente
    features = data.values  # Extrair as features como matriz numpy
    return features, labels

# Treinar e avaliar o modelo
def train_evaluate_model(config, X_train, y_train, X_val, y_val):
    model = MLP(input_dim=X_train.shape[1],
                output_dim=10,
                hidden_units=config['hidden_units'],
                dropout_rate=config['dropout_rate'],
                activations=config['activations'])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=config['learning_rate']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=config['batch_size'],
        epochs=config['epochs'],
        callbacks=[early_stopping],
        verbose=0
    )
    
    return max(history.history['val_accuracy']) 

## Cross-validation, apenas uma iteração (1 fold)
def cross_validate_model(config, files):
    # Apenas o primeiro fold para validação
    fold_number = 0
    X_val, y_val = load_fold_data(fold_number, files)
    X_train, y_train = [], []
    
    # Treino com os outros folds
    for i in range(len(files)):
        if i != fold_number:
            X_temp, y_temp = load_fold_data(i, files)
            X_train.append(X_temp)
            y_train.append(y_temp)
    
    X_train = np.concatenate(X_train, axis=0)
    y_train = np.concatenate(y_train, axis=0)
    
    # Treinar e avaliar para este fold
    accuracy = train_evaluate_model(config, X_train, y_train, X_val, y_val)
    return accuracy  # Retorna a acurácia deste único fold


# Função para avaliação em paralelo
def evaluate_config_parallel(args):
    config, files = args
    accuracy = cross_validate_model(config, files)
    print(f"Configuration: {config} | Accuracy: {accuracy}")
    return config, accuracy

# Definições de hiperparâmetros
configurations = {
    "hidden_units": [config[0] for config in activation_configs],
    "activations": [config[1] for config in activation_configs],
    "dropout_rate": [0, 0.1, 0.2, 0.3, 0.4, 0.5],
    "batch_size": [32, 64],
    "epochs": 1,
    "learning_rate": [0.001, 0.0001],
}


files = [f'datasets/urbansounds_features_fold{i}.csv' for i in range(1,11)] 

# Gerar todas as combinações de configurações
all_configs = generate_configs(configurations)

# Rodar tuning em paralelo
if __name__ == '__main__':
    num_workers = 8
    with Pool(num_workers) as pool:
        results = pool.map(evaluate_config_parallel, [(config, files) for config in all_configs])

    # Encontrar a melhor configuração
    best_config, best_accuracy = max(results, key=lambda x: x[1])
    print(f"Best configuration: {best_config}, Best accuracy: {best_accuracy}")

Configuration: {'hidden_units': [128, 64, 32], 'activations': ('relu', 'relu', 'relu'), 'dropout_rate': 0, 'batch_size': 32, 'epochs': 20, 'learning_rate': 0.001} | Accuracy: 0.6380297541618347
Configuration: {'hidden_units': [128, 64, 32], 'activations': ('relu', 'relu', 'relu'), 'dropout_rate': 0, 'batch_size': 32, 'epochs': 20, 'learning_rate': 0.0001} | Accuracy: 0.6529209613800049
Configuration: {'hidden_units': [128, 64, 32], 'activations': ('relu', 'relu', 'relu'), 'dropout_rate': 0, 'batch_size': 32, 'epochs': 50, 'learning_rate': 0.001} | Accuracy: 0.6231386065483093
Configuration: {'hidden_units': [128, 64, 32], 'activations': ('relu', 'sigmoid', 'tanh', 'relu'), 'dropout_rate': 0, 'batch_size': 32, 'epochs': 20, 'learning_rate': 0.001} | Accuracy: 0.6895761489868164
Configuration: {'hidden_units': [128, 64, 32], 'activations': ('relu', 'sigmoid', 'tanh', 'relu'), 'dropout_rate': 0, 'batch_size': 32, 'epochs': 20, 'learning_rate': 0.0001} | Accuracy: 0.6219931244850159
Config

2024-11-27 16:46:42.244136: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [256,128] vs. [0]
	 [[{{function_node __inference_one_step_on_data_515094}}{{node adam/Mul_11}}]]


Configuration: {'hidden_units': [256, 128, 64], 'activations': ('relu', 'sigmoid', 'tanh', 'sigmoid'), 'dropout_rate': 0.1, 'batch_size': 32, 'epochs': 20, 'learning_rate': 0.001} | Accuracy: 0.674685001373291
Configuration: {'hidden_units': [128, 64, 32], 'activations': ('relu', 'relu', 'sigmoid'), 'dropout_rate': 0.1, 'batch_size': 64, 'epochs': 50, 'learning_rate': 0.0001} | Accuracy: 0.6414662003517151


2024-11-27 16:46:44.639847: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at strided_slice_op.cc:117 : INVALID_ARGUMENT: Expected begin, end, and strides to be 1D equal size tensors, but got shapes [0], [1], and [1] instead.
2024-11-27 16:46:44.640134: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Expected begin, end, and strides to be 1D equal size tensors, but got shapes [0], [1], and [1] instead.
	 [[{{function_node __inference_one_step_on_data_616638}}{{node strided_slice}}]]


Configuration: {'hidden_units': [256, 128, 64], 'activations': ('relu', 'relu', 'sigmoid'), 'dropout_rate': 0.1, 'batch_size': 32, 'epochs': 50, 'learning_rate': 0.0001} | Accuracy: 0.6666666865348816
Configuration: {'hidden_units': [256, 128, 64], 'activations': ('relu', 'relu', 'relu'), 'dropout_rate': 0, 'batch_size': 32, 'epochs': 20, 'learning_rate': 0.001} | Accuracy: 0.6643757224082947
Configuration: {'hidden_units': [128, 64, 32], 'activations': ('relu', 'sigmoid', 'tanh', 'sigmoid'), 'dropout_rate': 0.1, 'batch_size': 32, 'epochs': 50, 'learning_rate': 0.0001} | Accuracy: 0.6563574075698853
Configuration: {'hidden_units': [256, 128, 64], 'activations': ('relu', 'relu', 'sigmoid'), 'dropout_rate': 0.1, 'batch_size': 64, 'epochs': 20, 'learning_rate': 0.001} | Accuracy: 0.6460481286048889
Configuration: {'hidden_units': [128, 64, 32], 'activations': ('relu', 'relu', 'sigmoid'), 'dropout_rate': 0.2, 'batch_size': 32, 'epochs': 20, 'learning_rate': 0.001} | Accuracy: 0.65750288963

2024-11-27 16:49:38.175938: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [32] vs. [0]
	 [[{{function_node __inference_one_step_on_data_770970}}{{node Equal}}]]


Configuration: {'hidden_units': [256, 128, 64], 'activations': ('relu', 'relu', 'sigmoid'), 'dropout_rate': 0.3, 'batch_size': 32, 'epochs': 20, 'learning_rate': 0.0001} | Accuracy: 0.6689575910568237
Configuration: {'hidden_units': [64, 32], 'activations': ('relu', 'sigmoid', 'tanh', 'relu'), 'dropout_rate': 0.1, 'batch_size': 64, 'epochs': 50, 'learning_rate': 0.001} | Accuracy: 0.6517754793167114
Configuration: {'hidden_units': [256, 128, 64], 'activations': ('relu', 'relu', 'relu'), 'dropout_rate': 0.2, 'batch_size': 64, 'epochs': 20, 'learning_rate': 0.0001} | Accuracy: 0.6895761489868164
Configuration: {'hidden_units': [128, 64, 32], 'activations': ('relu', 'relu', 'sigmoid'), 'dropout_rate': 0.3, 'batch_size': 64, 'epochs': 50, 'learning_rate': 0.0001} | Accuracy: 0.6357388496398926
Configuration: {'hidden_units': [256, 128, 64], 'activations': ('relu', 'relu', 'sigmoid'), 'dropout_rate': 0.3, 'batch_size': 32, 'epochs': 50, 'learning_rate': 0.001} | Accuracy: 0.674685001373291


2024-11-27 17:49:41.995125: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at strided_slice_op.cc:117 : INVALID_ARGUMENT: Expected begin, end, and strides to be 1D equal size tensors, but got shapes [0], [1], and [1] instead.
2024-11-27 17:49:41.996551: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Expected begin, end, and strides to be 1D equal size tensors, but got shapes [0], [1], and [1] instead.
	 [[{{function_node __inference_one_step_on_data_1940764}}{{node strided_slice}}]]


Configuration: {'hidden_units': [256, 128, 64], 'activations': ('relu', 'relu', 'elu'), 'dropout_rate': 0.5, 'batch_size': 32, 'epochs': 50, 'learning_rate': 0.001} | Accuracy: 0.648339033126831
Configuration: {'hidden_units': [128, 64, 32], 'activations': ('relu', 'sigmoid', 'relu'), 'dropout_rate': 0, 'batch_size': 32, 'epochs': 20, 'learning_rate': 0.001} | Accuracy: 0.6769759654998779
Configuration: {'hidden_units': [256, 128, 64], 'activations': ('relu', 'relu', 'tanh'), 'dropout_rate': 0.3, 'batch_size': 64, 'epochs': 20, 'learning_rate': 0.0001} | Accuracy: 0.6827033162117004
Configuration: {'hidden_units': [128, 64, 32], 'activations': ('relu', 'sigmoid', 'tanh', 'elu'), 'dropout_rate': 0.2, 'batch_size': 64, 'epochs': 20, 'learning_rate': 0.0001} | Accuracy: 0.5899198055267334
Configuration: {'hidden_units': [256, 128, 64], 'activations': ('relu', 'sigmoid', 'tanh', 'elu'), 'dropout_rate': 0.3, 'batch_size': 32, 'epochs': 50, 'learning_rate': 0.0001} | Accuracy: 0.681557834148

KeyboardInterrupt: 

eu tb quero testar as diferentes possibilidades de funções de ativação. ou seja eu sei que de uma camada para outra camada nos conseguimos fazer diferentes funções de ativação. consegues adicionar ao dicionário funções de ativação. combinações delas. 

atencao que depende do numero de camadas tb 
o numero de funções de ativação diferentes difere do numero de camadas 


mais técnicas de Regularization techniques to adopt ( por exemplo weight regularization, data augmen- tation, etc.)

há alguma forme de conseguir mudar o meu codigo de forma a ter isso? 

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import itertools
from pathos.multiprocessing import Pool

class MLP(tf.keras.Model):
    def __init__(self, input_dim, output_dim, hidden_units, dropout_rate, activations, regularization_type=None, regularization_value=0.01):
        super(MLP, self).__init__()
        self.hidden_layers = []
        self.regularization_type = regularization_type
        self.regularization_value = regularization_value
        
        for units, activation in zip(hidden_units, activations):
            self.hidden_layers.append(
                tf.keras.layers.Dense(units, activation=activation)
            )
            self.hidden_layers.append(tf.keras.layers.Dropout(dropout_rate))
        self.output_layer = tf.keras.layers.Dense(output_dim, activation='softmax')  # Classificação multi-classe

    def call(self, inputs):
        x = inputs
        for layer in self.hidden_layers:
            x = layer(x)
        return self.output_layer(x)
    
    def compute_regularization_loss(self):
        regularization_loss = 0.0
        if self.regularization_type:
            for layer in self.hidden_layers:
                if isinstance(layer, tf.keras.layers.Dense):
                    weights = layer.kernel
                    if self.regularization_type == 'l1':
                        regularization_loss += tf.reduce_sum(tf.abs(weights)) * self.regularization_value
                    elif self.regularization_type == 'l2':
                        regularization_loss += tf.reduce_sum(tf.square(weights)) * self.regularization_value
        return regularization_loss

    
    
def generate_configs(configurations):
    keys, values = zip(*configurations.items())
    return [dict(zip(keys, v)) for v in itertools.product(*values)]

# Gerar todas as combinações de hiperparâmetros
def generate_activation_configs(hidden_units_list, activation_functions):
    activation_configs = []
    for hidden_units in hidden_units_list:
        # Gerar combinações de funções de ativação do mesmo tamanho das camadas ocultas
        activations = list(itertools.product(activation_functions, repeat=len(hidden_units)))
        activation_configs.extend([(hidden_units, activation) for activation in activations])
    return activation_configs

hidden_units_list = [[128, 64, 32], [256, 128, 64], [64, 32], [512, 256, 128], [256, 128, 64, 32]]
activation_functions = ['relu', 'sigmoid', 'tanh']  # Funções de ativação possíveis

# Combinar hidden_units com ativations
activation_configs = generate_activation_configs(hidden_units_list, activation_functions)


# Função para carregar os dados de um fold específico
def load_fold_data(fold_number, files):
    data = pd.read_csv(files[fold_number])
    labels = data.pop('Label').values  # Extrair os rótulos diretamente
    features = data.values  # Extrair as features como matriz numpy
    return features, labels

# Treinar e avaliar o modelo
def train_evaluate_model(config, X_train, y_train, X_val, y_val):
    model = MLP(
        input_dim=X_train.shape[1],
        output_dim=10,
        hidden_units=config['hidden_units'],
        dropout_rate=config['dropout_rate'],
        activations=config['activations'],
        regularization_type=config.get('regularization_type', None),
        regularization_value=config.get('regularization_value', 0.01)
    )
    
    # Função de perda com regularização
    def loss_with_regularization(y_true, y_pred):
        base_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
        regularization_loss = model.compute_regularization_loss()
        return base_loss + regularization_loss

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=config['learning_rate']),
        loss=loss_with_regularization,
        metrics=['accuracy']
    )
    
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=config['batch_size'],
        epochs=config['epochs'],
        callbacks=[early_stopping],
        verbose=0
    )
    
    return max(history.history['val_accuracy'])


## Cross-validation, apenas uma iteração (1 fold)
def cross_validate_model(config, files):
    # Apenas o primeiro fold para validação
    fold_number = 0
    X_val, y_val = load_fold_data(fold_number, files)
    X_train, y_train = [], []
    
    # Treino com os outros folds
    for i in range(len(files)):
        if i != fold_number:
            X_temp, y_temp = load_fold_data(i, files)
            X_train.append(X_temp)
            y_train.append(y_temp)
    
    X_train = np.concatenate(X_train, axis=0)
    y_train = np.concatenate(y_train, axis=0)
    
    # Treinar e avaliar para este fold
    accuracy = train_evaluate_model(config, X_train, y_train, X_val, y_val)
    return accuracy  # Retorna a acurácia deste único fold


# Função para avaliação em paralelo
def evaluate_config_parallel(args):
    config, files = args
    accuracy = cross_validate_model(config, files)
    print(f"Configuration: {config} | Accuracy: {accuracy}")
    return config, accuracy

# Definições de hiperparâmetros
configurations = {
    "hidden_units": [[128, 64, 32], [256, 128, 64]],  # Exemplo de configurações
    "activations": [['relu', 'relu', 'relu'], ['tanh', 'tanh', 'tanh']],
    "dropout_rate": [0, 0.2],
    "batch_size": [32],
    "epochs": [5],
    "learning_rate": [0.001],
    "regularization_type": [None, 'l1', 'l2'],  # Sem regularização, L1, L2
    "regularization_value": [0.01, 0.001],      # Peso da regularização
}

files = [f'datasets/urbansounds_features_fold{i}.csv' for i in range(1,11)] 

# Gerar todas as combinações de configurações
all_configs = generate_configs(configurations)

# Rodar tuning em paralelo
if __name__ == '__main__':
    num_workers = 8
    with Pool(num_workers) as pool:
        results = pool.map(evaluate_config_parallel, [(config, files) for config in all_configs])

    # Encontrar a melhor configuração
    best_config, best_accuracy = max(results, key=lambda x: x[1])
    print(f"Best configuration: {best_config}, Best accuracy: {best_accuracy}")

/Users/franciscamihalache/Library/Python/3.9/lib/python/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 2.0.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Developer/CommandLineTools/Libra

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/franciscamihalache/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File 

AttributeError: _ARRAY_API not found

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [2]:
import tensorflow as tf
print(tf.__version__)


2.18.0
